In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Задание 1

1. Сгенерируйте 3 пары выборок заданного объема из разных законов распределения (два нормальных распределения, нормальное и отличное от нормального, два отличных от нормального распределения). Объем и законы распределения выбрать самостоятельно.

In [9]:
sample_size = 100
alpha = 0.05

# Генерируем выборки
sample_1 = np.random.normal(10, 2, size=sample_size)
sample_2 = np.random.normal(15, 2, size=sample_size)

2. Выдвинуть и проверить на заданном уровне значимости 𝛼 для каждой пары выборок гипотезы:
- о равенстве двух дисперсий 𝐻: $𝜎_1 = 𝜎_2$
- о равенстве двух выборочных средних 𝐻: $𝑚_1 = 𝑚_2$

Выбор критериев должен быть обусловлен видом распределения исходных выборок (проверить факт нормальности распределения выборок с 
помощью одного из критериев согласия, например Шапиро-Уилка) и равенством дисперсий (для выбора критерия для проверки гипотез о математических ожиданиях).

In [14]:
def check_norm_distribution(sample):
    _, p_value = stats.shapiro(sample)
    if p_value > alpha:
        print(f'Гипотеза о нормальности принимается')
    else:
        print(f'Гипотеза о нормальности отклоняется')
        
def F_test(s1, s2):
    # Проверка равенства дисперсий по F-критерию        
    var_1 = np.var(s1, ddof=1)
    var_2 = np.var(s2, ddof=1)

    print(f'Оценка дисперсии 1: {var_1:.4f}')
    print(f'Оценка дисперсии 2: {var_2:.4f}')

    if var_1 > var_2:
        F_score = var_1 / var_2
    else:
        F_score = var_2 / var_1
    print(f'Значение статистики: {F_score:.4f}')

    pvalue = 1 - stats.f.cdf(F_score, sample_size - 1, sample_size - 1)
    print(f'Значение p-value: {pvalue:.4f}')

    if pvalue > alpha:
        print(f'Гипотеза о равенстве дисперсий по F-критерию принимается')
    else:
        print(f'Гипотеза о равенстве дисперсий по F-критерию отклоняется')
        
def ttest_test(s1, s2, equal_var=True):
    _, p_value_ttest = stats.ttest_ind(s1, s2, equal_var=equal_var)
    print(f'Оценка выборочных средних: {p_value_ttest:.4f}')

    if p_value_ttest > alpha:
        print('Гипотеза о равенстве выборочных средних по t-критерию принимается')
    else:
        print('Гипотеза о равенстве выборочных средних по t-критерию отклоняется')
    print("\n")

def mannwhitheyu_test(s1, s2):
    _, p_value_mannwhitney = stats.mannwhitneyu(s1, s2)
    print(f'Оценка выборочных средних: {p_value_mannwhitney:.4f}')
    
    if p_value_mannwhitney > alpha:
        print('Гипотеза о равенстве выборочных средних по критерию Манна-Уитни принимается')
    else:
        print('Гипотеза о равенстве выборочных средних по критерию Манна-Уитни отклоняется')
    print("\n")
    
def fligner_test(s1, s2):
    # Проверка равенства дисперсий по критерию Флигнера
    _, p_value_fligner = stats.fligner(s1, s2)
    if p_value_fligner > alpha:
        print('Гипотеза о равенстве дисперсий по критерию Флигнера принимается')
    else:
        print('Гипотеза о равенстве дисперсий по критерию Флигнера отклоняется')

In [15]:
check_norm_distribution(sample_1)
check_norm_distribution(sample_2)

Гипотеза о нормальности принимается
Гипотеза о нормальности принимается


In [16]:
F_test(sample_1, sample_2)

Оценка дисперсии 1: 3.5993
Оценка дисперсии 2: 4.3383
Значение статистики: 1.2053
Значение p-value: 0.1772
Гипотеза о равенстве дисперсий по F-критерию принимается


In [17]:
ttest_test(sample_1, sample_2, equal_var=True)

Оценка выборочных средних: 0.0000
Гипотеза о равенстве выборочных средних по t-критерию отклоняется




## Задание 2

1. 1000 раз сгенерируйте выборки из нормального и любого отличного от нормального закона распределения для различных N. 
2. Примените критерии: Anderson-Darling test, Lilliefors test, Kolmogorov-Smirnov test и Shapiro-Wilk test. 
3. Сравните их мощности.

Мощность критерия – это вероятность того, что нулевая гипотеза будет отвергнута, когда альтернативная гипотеза верна. Правило проверки для основной гипотезы у нас следующее: если значение **p_value** больше уровня значимости, то основная гипотеза принимается. Значит нам нужно считать случаи, когда **p_value < alpha**.

In [7]:
def calculate_sw_power(n, alpha=0.05, iterations=1000, distribution=stats.norm, cretiorion=stats.shapiro,**dist_params):
    power = 0
    for _ in range(iterations):
        # Генерируем выборку из заданного распределения
        sample = distribution.rvs(size=n, **dist_params)
        # Применяем критерий Шапиро-Уилка
        _, p_value = cretiorion(sample)
        if p_value < alpha:
            power += 1
    return power / iterations

In [8]:
print("n = 10: ", calculate_sw_power(n=10, distribution=stats.expon))
print("n = 50: ", calculate_sw_power(n=50, distribution=stats.expon))
print("n = 100: ", calculate_sw_power(n=100, distribution=stats.expon))
print("n = 1000: ", calculate_sw_power(n=1000, distribution=stats.expon))

n = 10:  0.452
n = 50:  0.999
n = 100:  1.0
n = 1000:  1.0
